In [1]:
import torch
import torch.nn as nn
import import_ipynb
from collections import OrderedDict
from CrAsppNet import CRASPPNet

C:\Users\ASUS\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\ASUS\anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\ASUS\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


importing Jupyter notebook from CrAsppNet.ipynb


In [2]:
class Conv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Conv, self).__init__()
        self.layers = OrderedDict([
                                   ('conv', nn.Conv2d(
                                       in_channels = in_channels,
                                       out_channels = out_channels,
                                       kernel_size = 7, 
                                       stride = 1,
                                       padding = 3,
                                       bias=False
                                   )),
                                   ('norm', nn.BatchNorm2d(out_channels)),
                                   ('relu', nn.ReLU(inplace=True)),          
        ])
        self.conv = nn.Sequential(self.layers)
  
    def forward(self, x):
        return self.conv(x)

In [3]:
class ASPPWrapper(nn.Sequential):
    def __init__(self, aspp_module, in_channels=256, out_channels=128):
        super(ASPPWrapper, self).__init__(
            aspp_module,
            nn.Conv2d(
                in_channels=in_channels, 
                out_channels=out_channels,
                kernel_size=7, 
                stride=1,
                padding=3,
                bias=False
                ),
                nn.BatchNorm2d(out_channels),
                nn.ReLU()
        )

In [4]:
class UpTransConv(nn.Module):
    def __init__(self, in_channels, out_channels, size=None, scale_factor=None):
        super(UpTransConv, self).__init__()
    
        self.layers = OrderedDict([
                               ('up_sample', nn.Upsample(
                                   size=size,
                                   scale_factor=scale_factor,
                                   mode='bilinear',
                                   align_corners=False
                               )),               
                               ('conv_trans', nn.ConvTranspose2d(
                                  in_channels=in_channels,
                                  out_channels=out_channels,
                                  kernel_size=7,
                                  stride=1,
                                  padding=3,
                                  bias=True
                               ))
        ])
        self.conv = nn.Sequential(self.layers)
  
    def forward(self, x):
        return self.conv(x)

In [5]:
class CRASPPReUNet(CRASPPNet):
    def __init__(self, img_dimwh=224, in_channels=3, out_channels=1, downs_features=[32, 64, 128], atrous_rates=[6, 12, 18], scale_factor=3):
        super(CRASPPReUNet, self).__init__(img_dimwh=img_dimwh, in_channels=in_channels, downs_features=downs_features, atrous_rates=atrous_rates)
        self.downs_features = downs_features

        self.aspp_wrapper = ASPPWrapper(
            self.aspp,
            in_channels=downs_features[-1] * 2,
            out_channels=downs_features[-1]
        )
        up_features = downs_features[-1] + downs_features[0]
        self.up = UpTransConv(
            in_channels=downs_features[-1],
            out_channels=up_features,
            scale_factor=scale_factor
        )
        self.segment_branch = nn.ModuleDict({
            'middle' : Conv(in_channels=up_features, out_channels=int(up_features/4)),
            'final' : nn.Sequential(
                UpTransConv(
                    in_channels=int(up_features/4),
                    out_channels=downs_features[0],
                    size=img_dimwh
                ),
                Conv(
                    in_channels=downs_features[0],
                    out_channels=downs_features[0]
                ),
                nn.Conv2d(
                  in_channels=downs_features[0], 
                  out_channels=out_channels,
                  kernel_size=1, 
                  bias=True
                )
            )
        })   
        self.bottom.linear=nn.Linear(
            in_features=self._cal_features_up__(img_dimwh, scale_factor)**2 * (int(up_features/4) + up_features),
            out_features=2
        )
    def _cal_features_up__(self, img_dimwh, scale_factor):
        globavgpool_dimwh = img_dimwh
        for i in range(len(self.downs_features)):
            globavgpool_dimwh = int(globavgpool_dimwh / 2 - 0.5)
        globavgpool_dimwh = globavgpool_dimwh * scale_factor
        globavgpool_dimwh = int(globavgpool_dimwh / 2)
        return globavgpool_dimwh

    def forward(self, x):
        x = self.downs(x)
        x = self.aspp_wrapper(x)
        x = self.up(x)
        x_middle = self.segment_branch['middle'](x)
        x_segmentation = self.segment_branch['final'](x_middle)
        x_concat = torch.cat((x, x_middle), dim=1)
        x_classification = self.bottom['globavgpool'](x_concat)
        x_classification = torch.flatten(x_classification, 1)
        x_classification = self.bottom['linear'](x_classification)
        return x_classification, x_segmentation

In [6]:
def test_crasppreu():
    x = torch.randn((16, 3, 224, 224))
    model = CRASPPReUNet()
    cls, seg = model(x)
    print(cls.shape),
    print(seg.shape)
     

In [7]:
if __name__ == '__main__':
    test_crasppreu()

torch.Size([16, 2])
torch.Size([16, 1, 224, 224])
